# Supyrvisor Demo: Automatic Function Retry with Error Fixing

The `supyrvisor` module provides decorators that automatically retry function calls with modified arguments when errors are detected. This is useful for:

- Automatically fixing numerical convergence issues
- Retrying failed API calls with adjusted parameters
- Handling edge cases that can be fixed algorithmically

This module provides two decorator patterns:
1. **`supervisor`**: Original pattern using args/kwargs
2. **`manager`**: Modern pattern using normalized argument dictionaries

In [11]:
import numpy as np
from pycse.supyrvisor import (
    supervisor,
    manager,
    check_result,
    check_exception,
    TooManyErrorsException,
)

## Example 1: Basic `supervisor` Usage - Fixing Convergence Issues

Let's create a function that solves a numerical problem but might need parameter adjustment.

In [12]:
def check_convergence(args, kwargs, result):
    """Check if the result converged, and if not, propose a fix."""
    tolerance, max_iterations = args[0], kwargs.get("max_iter", 100)

    # If result is not accurate enough, try with smaller tolerance
    if abs(result["error"]) > 1e-3:
        new_tolerance = tolerance / 2
        new_max_iter = max_iterations * 2
        print(
            f"Convergence issue detected. Trying with tolerance={new_tolerance}, max_iter={new_max_iter}"
        )
        return (new_tolerance,), {"max_iter": new_max_iter}

    return None  # All good, no need to rerun


# Need max_errors=4 to get tolerance below 1e-4 (0.1 -> 0.05 -> 0.025 -> 0.0125 -> 0.00625)
@supervisor(check_funcs=[check_convergence], max_errors=10, verbose=True)
def solve_iteratively(tolerance, max_iter=100):
    """Simulate an iterative solver."""
    # Simulate that we need small tolerance to converge
    if tolerance < 1e-4:
        error = tolerance / 10  # Good convergence
    else:
        error = tolerance * 2  # Poor convergence

    return {"error": error, "iterations": max_iter, "tolerance": tolerance}


# This will automatically retry with better parameters
result = solve_iteratively(0.005)
print(f"\nFinal result: {result}")

Convergence issue detected. Trying with tolerance=0.0025, max_iter=200
Proposed fix in check_convergence: ((0.0025,), {'max_iter': 200})
Convergence issue detected. Trying with tolerance=0.00125, max_iter=400
Proposed fix in check_convergence: ((0.00125,), {'max_iter': 400})
Convergence issue detected. Trying with tolerance=0.000625, max_iter=800
Proposed fix in check_convergence: ((0.000625,), {'max_iter': 800})
Convergence issue detected. Trying with tolerance=0.0003125, max_iter=1600
Proposed fix in check_convergence: ((0.0003125,), {'max_iter': 1600})

Final result: {'error': 0.000625, 'iterations': 1600, 'tolerance': 0.0003125}


## Example 2: Exception Handling with `supervisor`

Let's handle exceptions that can be fixed by adjusting parameters.

In [13]:
def fix_division_by_zero(args, kwargs, exception):
    """Fix division by zero by adjusting the denominator."""
    if isinstance(exception, ZeroDivisionError):
        x, y = args
        print(f"Caught ZeroDivisionError! Adjusting y from {y} to 0.001")
        return (x, 0.001), {}
    return None


@supervisor(exception_funcs=[fix_division_by_zero], max_errors=5, verbose=True)
def compute_sqrt_ratio(x, y):
    """Compute sqrt(x/y) - may fail with division by zero."""
    if y == 0:
        raise ZeroDivisionError("Cannot divide by zero")
    result = np.sqrt(x / y)
    return result


# Test with division by zero
print("Test 1: Division by zero")
result1 = compute_sqrt_ratio(10, 0)
print(f"Result: {result1}\n")

# Test with normal values
print("Test 2: Normal computation")
result2 = compute_sqrt_ratio(16, 4)
print(f"Result: {result2}")

Test 1: Division by zero
Caught ZeroDivisionError! Adjusting y from 0 to 0.001
Proposed fix in fix_division_by_zero: ((10, 0.001), {})
Result: 100.0

Test 2: Normal computation
Result: 2.0


## Example 3: Modern `manager` Pattern with Checker Decorators

The `manager` decorator uses a cleaner pattern with `@check_result` and `@check_exception` decorators.

In [14]:
@check_result
def check_positive_result(arguments, result):
    """Ensure result is positive, otherwise increase the offset."""
    if result < 0:
        current_offset = arguments.get("offset", 0)
        new_offset = current_offset + 5
        print(f"Negative result {result}. Increasing offset to {new_offset}")
        new_args = arguments.copy()
        new_args["offset"] = new_offset
        return new_args
    return None


@manager(checkers=[check_positive_result], max_errors=5, verbose=True)
def compute_value(x, offset=0, scale=1.0):
    """Compute a scaled and offset value."""
    result = (x + offset) * scale
    print(f"  Computing: ({x} + {offset}) * {scale} = {result}")
    return result


# This will automatically adjust offset until result is positive
result = compute_value(-8, offset=0)
print(f"\nFinal result: {result}")

  Computing: (-8 + 0) * 1.0 = -8.0
Negative result -8.0. Increasing offset to 5
Proposed fix in wrapper: {'x': -8, 'offset': 5, 'scale': 1.0}
  Computing: (-8 + 5) * 1.0 = -3.0
Negative result -3.0. Increasing offset to 10
Proposed fix in wrapper: {'x': -8, 'offset': 10, 'scale': 1.0}
  Computing: (-8 + 10) * 1.0 = 2.0

Final result: 2.0


## Example 4: File Processing with Automatic Error Recovery

In [15]:
import json
import ast


@check_exception
def fix_json_decode_error(arguments, exception):
    """If JSON parsing fails, try alternative parsing strategies."""
    if isinstance(exception, json.JSONDecodeError):
        # Try with a more lenient approach using ast.literal_eval
        new_args = arguments.copy()
        new_args["use_ast"] = True
        print("JSON decode error. Retrying with ast.literal_eval")
        return new_args
    return None


@check_result
def validate_data(arguments, result):
    """Validate the parsed data has required fields."""
    if result is not None and "required_field" not in result:
        # Add default value and reprocess
        print("Missing required_field. Adding default value.")
        new_args = arguments.copy()
        new_args["add_defaults"] = True
        return new_args
    return None


@manager(checkers=[fix_json_decode_error, validate_data], max_errors=3, verbose=True)
def load_config(data_str, use_ast=False, add_defaults=False):
    """Load configuration from JSON string."""
    if use_ast:
        # More lenient parsing using ast.literal_eval
        parsed = ast.literal_eval(data_str)
    else:
        parsed = json.loads(data_str)

    if add_defaults:
        parsed.setdefault("required_field", "default_value")

    return parsed


# Test with invalid JSON that becomes valid with ast.literal_eval
malformed_json = "{'name': 'test', 'value': 42}"  # Single quotes, not valid JSON
result = load_config(malformed_json)
print(f"\nParsed result: {result}")

JSON decode error. Retrying with ast.literal_eval
Proposed fix in wrapper: {'data_str': "{'name': 'test', 'value': 42}", 'use_ast': True, 'add_defaults': False}
Missing required_field. Adding default value.
Proposed fix in wrapper: {'data_str': "{'name': 'test', 'value': 42}", 'use_ast': True, 'add_defaults': True}

Parsed result: {'name': 'test', 'value': 42, 'required_field': 'default_value'}


## Example 5: Numerical Optimization with Adaptive Parameters

In [16]:
from scipy.optimize import minimize


@check_result
def check_optimization_success(arguments, result):
    """Check if optimization succeeded, adjust method if not."""
    if not result.success:
        current_method = arguments.get("method", "BFGS")

        # Try different methods in sequence
        method_sequence = ["BFGS", "Nelder-Mead", "Powell", "CG"]

        try:
            current_idx = method_sequence.index(current_method)
            if current_idx < len(method_sequence) - 1:
                new_method = method_sequence[current_idx + 1]
                print(f"Optimization failed with {current_method}. Trying {new_method}")
                new_args = arguments.copy()
                new_args["method"] = new_method
                return new_args
        except ValueError:
            pass

    return None


@manager(checkers=[check_optimization_success], max_errors=4, verbose=True)
def optimize_function(x0, method="BFGS"):
    """Optimize a test function."""

    # Rosenbrock function - difficult to optimize
    def rosenbrock(x):
        return (1 - x[0]) ** 2 + 100 * (x[1] - x[0] ** 2) ** 2

    print(f"  Optimizing with method: {method}")
    result = minimize(
        rosenbrock, x0, method=method, options={"maxiter": 10}
    )  # Low maxiter to force failures
    return result


# This will try different optimization methods until one succeeds
result = optimize_function([0, 0])
print("\nOptimization result:")
print(f"  Success: {result.success}")
print(f"  Final x: {result.x}")
print(f"  Function value: {result.fun}")

  Optimizing with method: BFGS
Optimization failed with BFGS. Trying Nelder-Mead
Proposed fix in wrapper: {'x0': [0, 0], 'method': 'Nelder-Mead'}
  Optimizing with method: Nelder-Mead
Optimization failed with Nelder-Mead. Trying Powell
Proposed fix in wrapper: {'x0': [0, 0], 'method': 'Powell'}
  Optimizing with method: Powell
Optimization failed with Powell. Trying CG
Proposed fix in wrapper: {'x0': [0, 0], 'method': 'CG'}
  Optimizing with method: CG

Optimization result:
  Success: False
  Final x: [0.79046814 0.6184045 ]
  Function value: 0.04804500539743658


## Example 6: Maximum Errors Limit

In [17]:
@check_result
def always_fail(arguments, result):
    """A checker that always proposes a fix (for demonstration)."""
    print("  Checker called, proposing fix...")
    new_args = arguments.copy()
    new_args["attempt"] = arguments.get("attempt", 0) + 1
    return new_args


@manager(checkers=[always_fail], max_errors=3, verbose=True)
def will_exceed_max_errors(attempt=0):
    """This function will hit max_errors limit."""
    print(f"  Attempt {attempt}")
    return f"attempt_{attempt}"


# This will raise TooManyErrorsException
try:
    result = will_exceed_max_errors()
except TooManyErrorsException as e:
    print(f"\nCaught expected exception: {e}")

  Attempt 0
  Checker called, proposing fix...
Proposed fix in wrapper: {'attempt': 1}
  Attempt 1
  Checker called, proposing fix...
Proposed fix in wrapper: {'attempt': 2}
  Attempt 2
  Checker called, proposing fix...
Proposed fix in wrapper: {'attempt': 3}

Caught expected exception: Maximum number of errors (3) reached


## Example 7: Combining Multiple Checkers

In [18]:
@check_result
def check_range(arguments, result):
    """Ensure result is in valid range."""
    if result < 0:
        new_args = arguments.copy()
        new_args["lower_bound"] = arguments.get("lower_bound", 0) + 1
        print(f"Result too low: {result}. Increasing lower_bound")
        return new_args
    elif result > 100:
        new_args = arguments.copy()
        new_args["upper_bound"] = arguments.get("upper_bound", 100) - 10
        print(f"Result too high: {result}. Decreasing upper_bound")
        return new_args
    return None


@check_result
def check_integer(arguments, result):
    """Ensure result is close to an integer."""
    if abs(result - round(result)) > 0.1:
        new_args = arguments.copy()
        new_args["force_integer"] = True
        print(f"Result not close to integer: {result}. Forcing integer output")
        return new_args
    return None


@manager(checkers=[check_range, check_integer], max_errors=5, verbose=True)
def compute_bounded_value(seed, lower_bound=0, upper_bound=100, force_integer=False):
    """Compute a value within bounds."""
    np.random.seed(seed)
    value = np.random.uniform(lower_bound, upper_bound)

    if force_integer:
        value = round(value)

    print(
        f"  Generated value: {value} (bounds: [{lower_bound}, {upper_bound}], integer: {force_integer})"
    )
    return value


# This will adjust bounds until result is in valid range and near integer
result = compute_bounded_value(42, lower_bound=-50, upper_bound=150)
print(f"\nFinal result: {result}")

  Generated value: 24.9080237694725 (bounds: [-50, 150], integer: False)

Final result: 24.9080237694725


## Summary

The `supyrvisor` module provides powerful automatic error recovery:

### Key Features:
- **Automatic retry**: Functions are automatically retried with modified parameters
- **Two patterns**: `supervisor` (args/kwargs) and `manager` (argument dict)
- **Flexible checking**: Separate handlers for results and exceptions
- **Safety limits**: `max_errors` prevents infinite loops
- **Debugging support**: `verbose=True` shows all retry attempts

### When to Use:
- Numerical algorithms that need parameter tuning
- API calls that may need retry with different parameters
- Data processing with fallback strategies
- Any function where errors can be fixed algorithmically

### Best Practices:
1. Always set `max_errors` to prevent infinite loops
2. Use `verbose=True` during development
3. Make checkers return `None` when no fix is needed
4. Keep checker logic simple and focused
5. Use `manager` pattern for new code (cleaner API)